In [4]:
!pip install yfinance numpy matplotlib ipywidgets
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, widgets

def delta_hedge_interactive(ticker="AAPL", strike_price_factor=5, expiration=""):
    stock = yf.Ticker(ticker)
    stock_data = stock.history(period="1mo")
    current_price = stock_data['Close'][-1]

    try:
        options = stock.option_chain(expiration)
        puts = options.puts
    except Exception as e:
        print(f"Error fetching options: {e}. Try a different expiration.")
        return

    strike_price = current_price * (1 - strike_price_factor/100)
    selected_put = puts.iloc[(puts['strike'] - strike_price).abs().argsort()[:1]].squeeze()

    if selected_put.empty:
        print("No put options found near the selected strike. Try a different factor/expiration.")
        return

    print(f"\n Selected Put Option:")
    print(f"- Strike Price: ${selected_put['strike']:.2f}")
    print(f"- Premium: ${selected_put['lastPrice']:.2f}")
    print(f"- Expiration: {expiration}")

    delta_put = -0.5
    hedge_ratio = -1 / delta_put
    num_puts_needed = int(hedge_ratio)
    print(f"\n Hedge Ratio: Buy {num_puts_needed} put contracts (each covers 100 shares).")

    simulated_prices = np.linspace(current_price * 0.7, current_price * 1.3, 100)
    unhedged_pnl = (simulated_prices - current_price) * 100
    put_payoff = np.maximum(selected_put['strike'] - simulated_prices, 0) - selected_put['lastPrice']
    hedged_pnl = unhedged_pnl + (put_payoff * 100 * num_puts_needed)

    plt.figure(figsize=(10, 6))
    plt.plot(simulated_prices, unhedged_pnl, label="Unhedged (Stock Only)", color='blue')
    plt.plot(simulated_prices, hedged_pnl, label="Hedged (Stock + Puts)", color='red')
    plt.axvline(current_price, color='black', linestyle='--', label="Current Price")
    plt.axhline(0, color='grey', linestyle=':')
    plt.xlabel("Stock Price at Expiration ($)")
    plt.ylabel("Profit/Loss ($)")
    plt.title(f"Delta Hedging: {ticker} (Strike = ${selected_put['strike']:.2f})")
    plt.legend()
    plt.grid()
    plt.show()

ticker_input = widgets.Text(description="Ticker:")
strike_factor_input = widgets.IntSlider(min=1, max=100, description="Strike % Below:")
expiration_input = widgets.Text(description="Expiration (YYYY-MM-DD):")

interact(
    delta_hedge_interactive,
    ticker=ticker_input,
    strike_price_factor=strike_factor_input,
    expiration=expiration_input
);

interactive(children=(Text(value='', description='Ticker:'), IntSlider(value=1, description='Strike % Below:',…